In [1]:
import numpy as np
import sympy
import control as cnt
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dp(expr):
    display(Math(vlatex(expr)))

# Finding the Lagrangian

defined as L = K - P, the difference between the kinetic and potential energy of the system

use sympy to create equations for K and P and compute

In [2]:
t = symbols('t') # defining t and theta/phi/other coords as functions of t is necessary to get proper EoMs

theta = symbols('\\theta', cls=Function)
theta = theta(t)
theta_dot = theta.diff(t)
theta_ddot = theta.diff(t,2)

b, g, ell, m, k1, k2, F, tau, s, theta_e = symbols('b, g, ell, m, k_1, k_2, F, \\tau, s, \\theta_e')

linear velocities...

In [3]:
q = Matrix([[ell*cos(theta)], [ell*sin(theta)], [0]]) # the z coord will be used by our angular velocity, here it's just 0
q_dot = q.diff(t)
dp(q_dot)
v = q_dot

<IPython.core.display.Math object>

now it's time for angular...

In [4]:
w = Matrix([[0], [0], [theta_dot]]) # z coord only, think cross products and rotational shtuff

# potential and kinetic energy

kinetic is given by $ K = {1/2 m_1 v_1^T v_1 + 1/2\omega_1^T J_1 \omega_1}$ for however many things you have

potential is $ P = {m_1 g h_1} $ + whatever springs are present

J matrices...

point mass: $ J = 0 $ :steamhappy:

thin rod: $ J = \begin{pmatrix}
0 & 0 & 0\\
0 & \frac{m l ^2}{12} & 0\\
0 & 0 & \frac{m l ^2}{12}
\end{pmatrix} $

thin plate: $ J = \begin{pmatrix}
\frac{m \omega ^2}{12} & 0 & 0\\
0 & \frac{m l ^2}{12} & 0\\
0 & 0 & \frac{m l ^2}{12} + \frac{m \omega ^2}{12}
\end{pmatrix} $

In [5]:
K_total = 1/2*m*v.T @ v
# dp(K_total)
K = 0.5*ell**2*m*(sin(theta)*theta_dot)**2 + .5*ell**2*m*(cos(theta)*theta_dot)**2
# dp(K)
P_spring = 1/2*k1*theta**2 + 1/4*k2*theta**4
P = m*g*ell*sin(theta) - P_spring
L = K - P
dp(L)

<IPython.core.display.Math object>

# lagrangian

q and q_dot are for our generalized coordinates (in the practice final it's just theta and theta_dot, for 2 gen coords it will be different)

In [6]:
# behold...
def derive_Lagrangian(L, q, q_dot):
    term_1 = (sympy.tensor.derive_by_array(L, q_dot)).diff(t)
    term_2 = sympy.tensor.derive_by_array(L, q)
    return term_1 - term_2

# def derive_Lagrangian(L, z, theta, z_dot, theta_dot):
#     term_1 = (sympy.tensor.derive_by_array(L, z_dot)).diff(t)
#     term_2 = sympy.tensor.derive_by_array(L, z)
#     term_3 = (sympy.tensor.derive_by_array(L, theta_dot)).diff(t)
#     term_4 = sympy.tensor.derive_by_array(L, theta)
#     ans = Matrix([term_1 - term_2, term_3 - term_4])
#     return ans

In [7]:
LG = derive_Lagrangian(L, theta, theta_dot)
dp(LG)

<IPython.core.display.Math object>

# Euler Lagrange

defined by 
# $ \frac{d}{dt} (\frac{\delta L (q, \dot{q})}{\delta{\dot{q}}}) - \frac{\delta L (q)}{\delta q} = \tau - B \dot{q}$

n = number of gen coords

$ \dot{q}$ is the dot for all n gen coords, should be n x 1

$ B $ is the damping force for each gen coord, should be n x n

$ \tau $ has a force for each spatial gen coord and a torque for each angular gen coord, should be n x 1 

In [8]:
LHS = Matrix([[derive_Lagrangian(L, theta, theta_dot)]])

Tau = Matrix([[tau]])
B = Matrix([[b]])
q_d = Matrix([[theta_dot]])
RHS = Tau - B @ q_d

dynamics = Eq(LHS, RHS) # this is also our euler lagrange eq
dp(dynamics)

<IPython.core.display.Math object>

# Equations of motion

taking $ \begin{pmatrix}
        \theta\\
        \dot{\theta}        
        \end{pmatrix} $ and making it into 
        $ \begin{pmatrix}
        \dot{\theta}\\
        \ddot{\theta}       
        \end{pmatrix} $

essentially we solve for $ \ddot{\theta} $ and plug it into another matrix with $ \dot{\theta} $

inside the `(theta_ddot,)` we can include however many variables we need to solve for, namely however many ddots are involved

In [9]:
solve_dict = solve(dynamics, (theta_ddot,), simplify=True, dict=True)[0]
# dp(solve_dict)

theta_ddot_expr = solve_dict[theta_ddot]

EoM = Matrix([theta_dot, theta_ddot_expr])
dp(EoM)

<IPython.core.display.Math object>

# Part 2
# Linearizing and stuff

first step of linearization is finding the equilibria, or where the system is at rest/balanced

get $ \theta_e $ and then find the $ \tau_e $ to maintain $ \theta_e $

at equilibrium $ \theta = 0 $ so $ \dot{\theta} $ and $ \ddot{\theta} $ also $ = 0 $ so we can plug in $ 0 $ for those and solve for tau   

In [10]:
EoM_eq = EoM.subs({theta_dot: 0, theta: theta_e})
tau_e = solve(EoM_eq.subs({theta_e: 0}), (tau,), simplify = True, dict = True)[0] # the subs here is to set theta_e = 0, it isn't done for EoM_eq for symbolic porpoises
dp(tau_e)

<IPython.core.display.Math object>

# Finding linearized state space eqs

these are in the form

$ \dot{x} = Ax + B\tilde{u} $

$ y = Cx + D\tilde{u} $

note: these are the un-equilibrized (not a word) equations because otherwise it won't work. there's probably a work around but it's 10pm and i don't want to work any harder on this than i have to. i will `.subs({theta: 0})` later on for the transfer function

In [11]:
theta_val = 0
m_val = 0.1
b_val = 0.1
ell_val = 0.25
g_val = 9.8
k1_val = 0.02
k2_val = 0.02
substitutions = {theta: theta_val, m: m_val, b: b_val, ell: ell_val, g: g_val, k1: k1_val, k2: k2_val} # makes .subs easier

In [12]:
# define state variables
state = Matrix([theta, theta_dot])

A = EoM.jacobian(state)

dp(A)

A_eq = A.subs({theta: 0})
dp(A_eq) # this is what 2.5 wants

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [13]:
inputs = Matrix([[tau]])

B = EoM_eq.jacobian(inputs)
dp(B)

B_eq = B # tau is not present in B so B_eq is the same thing

<IPython.core.display.Math object>

We still need C and D equations, 

$ y = Cx + Du $

$C$ is used to select the variables present in your output $y$, it will be a 1 x n where $x$ is a n x 1

$D$ is normally zero... so stick with that I guess

In [14]:
C = Matrix([[1, 0]])
D = Matrix([0])

# C_eq and D_eq aren't going to be any different...

we can use these matrices to write our linearized state space equations, and find our transfer function

Now we get to use the control library which makes life 10000000000000000x better!!!!

plug in real values into your A B C and D matrices and pass them into `cnt.StateSpace()`

In [15]:
A_sub = A.subs(substitutions)

B_sub = B.subs(substitutions)

C_sub = C

D_sub = D

rodmass = cnt.StateSpace(A_sub, B_sub, C_sub, D_sub, False) # boolean tells cnt whether or not it's a discrete system. F = continuous, T = discrete

now do `cnt.ss2tf` (state space to transfer func)

In [16]:
cnt.ss2tf(rodmass)

TransferFunction(array([3.55271368e-15, 1.60000000e+02]), array([ 1. , 16. , -3.2]))

i'm not sure why that s term is in the top. it's  .0000000000000035553 so we're just going to ignore it... ok?

we can also put the entire system in canonical observable or controller form with a single function

we find our transfer function symbolically with the following equation:

$ H(s) = C(sI-A)^{-1}B+D $

$I$ is n x n (same as $A$)

In [17]:
I = eye(2)

H = C.subs({theta: 0}) @ (s*I - A.subs({theta: 0})).inv() @ B.subs({theta: 0}) + D.subs({theta: 0})

dp(H)

<IPython.core.display.Math object>

add a kd*s and kp to the denominator, then divide by ml**2 resultant fraction for gain stuff should be:

In [20]:
kp, kd = symbols('k_p, k_d')
top = 1/(m*ell**2)
bottom = (kp + k1)/(m*ell**2) + s*(kd + b)/(m*ell**2) + s**2

tf_gains = top/bottom
tf_gains_sub = tf_gains.subs(substitutions)
dp(tf_gains_sub)

<IPython.core.display.Math object>

# Finding the gains

In [22]:
wn = symbols('\\omega_n')

zeta_val = 0.707
kp_val = 3/(rad(20))

wn_val = sqrt((kp_val + k1_val)/(m_val*ell_val**2))

kd_eqtn = Eq(2 * wn * zeta_val,(b+kd)/(m*ell**2)) # change the RHS to 

kd_val = solve(kd_eqtn.subs({
    m: 0.1, 
    ell: 0.25, 
    zeta_val: .707,
    wn: wn_val,
    b: 0.1
}), kd)[0]

round(kd_val, 3)

# tf_gains_sub.subs({kp: round(kp_val,3)})

with this we can directly solve for kd and wn

In [42]:
cnt.canonical_form(rodmass, 'reachable') 

# the string tells it what canonical form to find.
# observable is observer canonical form
# reachable is controller canonical form
# modal is some other thing we never went over, don't use it

(StateSpace(array([[-16. ,   3.2],
        [  1. ,   0. ]]), array([[1.],
        [0.]]), array([[  0., 160.]]), array([[0.]])),
 array([[2.22044605e-17, 6.25000000e-03],
        [6.25000000e-03, 0.00000000e+00]]))

there are once again really tiny values in this, no doubt from the $ 3.553 \times 10^{-15} $ term... i'm just evaluating it at 0

producing controllability and observability matrices

In [ ]:
cnt.ctrb(A_sub, B_sub)
cnt.obsv(A_sub, B_sub)

# PID and PD Controllers

In [ ]:
# PD setup

def __init__(self):
        tr = 2 # tune this
        zeta = 0.707

        # desired natural frequency
        wn = 2.2 / tr
        alpha1 = 2.0 * zeta * wn
        alpha0 = wn**2

        b0 = 
        a0 =            # solve for these
        a1 =
        self.kp = (alpha0 - a0) / b0
        self.kd = (alpha1 - a1) / b0
        # PD gains
        print('kp: ', self.kp)
        print('kd: ', self.kd)

alternative init method with poles instead of wn, tr, and zeta

In [ ]:
    def __init__(self):
        # Declare pole locations
        p1 = -1
        p2 = -1.5
        alpha0 = (-p1) * (-p2) # Coefficient in front of s^0 in desired characteristic equation
        alpha1 = (-p1) + (-p2) # Coefficient in front of s^1 in desired characteristic equation
        b0 = 1/P.m
        a0 = P.k/P.m
        a1 = P.b/P.m
        self.kp = (alpha0 - a0) / b0
        self.kd = (alpha1 - a1) / b0
        # PD gains
        print('kp: ', self.kp)
        print('kd: ', self.kd)

In [ ]:
# PD update functions have the form:

def update(self, theta_r, x):
        theta = x[0][0]
        thetadot = x[1][0]
        # compute the linearized torque using PD control
        tau_tilde = self.kp * (theta_r - theta) - self.kd * thetadot

        tau_sat = saturate(tau_tilde, P.F_max)
        return tau_sat

# saturation isn't as

In [ ]:
# PID setup
def __init__(self):
        #  tuning parameters
        tr = 2.0
        zeta = 0.707
        self.ki = 1.5  # integrator gain, tune this

        # desired natural frequency
        wn = 2.2 / tr
        alpha1 = 2.0 * zeta * wn
        alpha0 = wn**2

        b0 = 
        a0 =            # solve for these
        a1 = 
        self.kp = (alpha0 - a0) / b0
        self.kd = (alpha1 - a1) / b0
        print('kp: ', self.kp)
        print('ki: ', self.ki)
        print('kd: ', self.kd)

        # dirty derivative gainss
        self.beta = (2.0 * P.sigma - P.Ts) / (2.0 * P.sigma + P.Ts)  # dirty derivative gain

        #----------------------------------------------------------
        # variables for integrator and differentiator
        self.z_dot = 0.0             # estimated derivative of y
        self.z_d1 = 0.0              # Signal y delayed by one sample
        self.error_dot = 0.0         # estimated derivative of error
        self.error_d1 = 0.0          # Error delayed by one sample
        self.integrator = 0.0        # integrator

In [ ]:
# PID updates are like this:
def update(self, z_r, y):
        z = y[0][0]
        
        # Compute the current error
        error = z_r - z

        # integrate error
        self.integrator = self.integrator + (P.Ts / 2) * (error + self.error_d1)

        # differentiate z
        self.z_dot = self.beta * self.z_dot + (1 - self.beta) * ((z - self.z_d1) / P.Ts)
        
        # PID control
        F_unsat = self.kp * error + self.ki * self.integrator - self.kd * self.z_dot
        F = saturate(F_unsat, P.F_max)

        # integrator anti - windup
        if self.ki != 0.0:
            self.integrator = self.integrator + P.Ts / self.ki * (F - F_unsat)
            
        # update delayed variables
        self.error_d1 = error
        self.z_d1 = z
        return F

In [ ]:
# saturation function in case you lose it

def saturate(self, u):
    if abs(u) > self.limit:
        u = self.limit*np.sign(u)
    return u

# also anti-windup just in case

if np.abs(self.h_dot) < 0.5:
    self.integrator_h = self.integrator_h + (P.Ts / 2) * (error_h + self.error_h_d1)



# observer controllers
these update both a PID controller and an observer so there's 2 update functions 

this code is for a disturbance observer. idk the difference between this and just a regular observer so... cool

In [ ]:
class ctrlObserver:
    def __init__(self):
        #--------------------------------------------------
        # State Feedback Control Design
        #--------------------------------------------------
        #  tuning parameters
        tr = 1.5
        zeta = 0.707
        integrator_pole = -1
        tr_obs = tr/10  # rise time frequency for observer
        zeta_obs = 0.707  # damping ratio for observer


        # State Space Equations
        # xdot = A*x + B*u
        # y = C*x
        self.A = np.array([[0.0, 0.0],
                           [0.0, 0.0]])
        self.B = np.array([[0.0],
                           [0.0]])       
        self.C = np.array([[0.0, 0.0]])
        # form augmented system
        A1 = np.vstack((np.hstack((self.A, np.zeros((2,1)))), # np.zeros matches A
                        np.hstack((-self.C, np.zeros((1,1)))) )) #np.zeros matches C
        B1 = np.vstack( (self.B, 0.0) )

        # gain calculation
        wn = 2.2 / tr  # natural frequency
        des_char_poly = np.convolve([1, 2*zeta*wn, wn**2],
                                    [1, -integrator_pole])
        des_poles = np.roots(des_char_poly)
        # Compute the gains if the system is controllable
        if np.linalg.matrix_rank(cnt.ctrb(A1, B1)) != 3: # rank will be n + 1, where A is a n x n matrix
            print("The system is not controllable")
        else:
            K1 = cnt.place(A1, B1, des_poles)
            self.K = K1[0][0:2]
            self.ki = K1[0][2]


        # observer design
        wn_obs = 2.2 / tr_obs
        des_obsv_char_poly = [1, 2*zeta_obs*wn_obs, wn_obs**2]
        des_obsv_poles = np.roots(des_obsv_char_poly)
        # Compute the gains if the system is controllable
        if np.linalg.matrix_rank(cnt.ctrb(self.A.T, self.C.T)) != 2:
            print("The system is not observerable")
        else:
            self.L = cnt.acker(self.A.T, self.C.T, des_obsv_poles).T
        print('K: ', self.K)
        print('ki ', self.ki)
        print('L^T: ', self.L.T)
        #--------------------------------------------------
        # variables to implement integrator
        self.integrator = 0.0  # integrator
        self.error_d1 = 0.0  # error signal delayed by 1 sample
        self.x_hat = np.array([
            [0.0],  # z_hat_0
            [0.0],  # zdot_hat_0
        ])
        self.tau_d1 = 0.0  # control torque, delayed 1 sample

    def update(self, z_r, y):
        # update the observer and extract z_hat
        x_hat = self.update_observer(y)
        z_hat = x_hat[0][0]
        # integrate error
        error = z_r - z_hat
        self.integrator = self.integrator \
                          + (P.Ts / 2.0) * (error + self.error_d1)
        self.error_d1 = error
        # Compute the state feedback controller
        f_tilde = -self.K @ x_hat - self.ki * self.integrator
        # compute total torque
        F = saturate(f_tilde[0], P.F_max)
        self.tau_d1 = F
        return F, x_hat

    def update_observer(self, y_m):
        # update the observer using RK4 integration
        F1 = self.observer_f(self.x_hat, y_m)
        F2 = self.observer_f(self.x_hat + P.Ts / 2 * F1, y_m)
        F3 = self.observer_f(self.x_hat + P.Ts / 2 * F2, y_m)
        F4 = self.observer_f(self.x_hat + P.Ts * F3, y_m)
        self.x_hat = self.x_hat + P.Ts / 6 * (F1 + 2*F2 + 2*F3 + F4)
        return self.x_hat

    def observer_f(self, x_hat, y_m):
        # xhatdot = A*(xhat-xe) + B*(u-ue) + L(y-C*xhat)
        xhat_dot = self.A @ x_hat\
                   + self.B * (self.tau_d1)\
                   + self.L * (y_m - self.C @ x_hat)
        return xhat_dot


def saturate(u, limit):
    if abs(u) > limit:
        u = limit * np.sign(u)
    return u

# finding feedback gains

the augmented feedback gain $K$ is defined by this equation:

$ K = (\alpha - \textbf{a}_A) \mathcal{A}_A^{-1} \mathcal{C}_{AB}^{-1}$

$\alpha$ is an array of the n-1, n-2, etc coefficients of $ \Delta_{cl}^{d} $ (desired close loop char polynom)

you can find $ \Delta_{cl}^{d} $ by foiling out the desired poles

$ \textbf{a}_A $ is an array of the n-1, n-2, n-3 etc coefficients of $ \Delta_{ol} $ (open loop char polynom) 

you can find $ \Delta_{ol} = \text{det} (sI - A)$

(skip the coefficients of the highest power s terms of the characteristic polynomials. that's what i mean by n-1, n-2 coeff but i want to be very clear)


$\mathcal{A}_A$ is a matrix of the form 

$ \begin{pmatrix}
1 & a & b\\
0 & 1 & a\\
0 & 0 & 1
\end{pmatrix} $

where $ a_A = \begin{pmatrix}
a & b & c
\end{pmatrix} $


you will have to `.inv` $ \mathcal{A}_A $ to get $\mathcal{A}_A^{-1}$


$\mathcal{C}_{AB}^{-1}$ is your controllability matrix. use `cnt.ctrb(A,B)` to find $\mathcal{C}_{AB}$ and then .inv it


# note

feedback gain that places poles at desired location is given by $ K_c = \alpha - \textbf{a}_A $ but this is only when in controller canonic form

# reference gain

$ k_r = \frac{-1}{C_r (A-BK)^{-1}B} $

$C_r$ is the $C$ matrix required to for the reference output, a la $y_r = C_r x$

# finding observer gains

augmented observer gain defined as 

$ L = \mathcal{O}_{A,C}^{-1} \mathcal{A}_{A}^{-T} (\beta - \textbf{a}_A)^{T} $

$ \mathcal{O}_{A,C} $ is your observability matrix, use `cnt.obsv(A,B)` to find it
$ \beta $ is an array of the n-1, n-2, etc coefficients of $ \Delta_{cl,obs}^{d} $ (desired close loop char polynom)

$ \Delta_{cl,obs}^{d} = \text{det}(sI - (A_o - L_o C_o))$

$ \textbf{a}_A $ is an array of the n-1, n-2, n-3 etc coefficients of $ \Delta_{ol} $ (open loop char polynom) 

$ \Delta_{ol} = \text{det}(sI - A_o) $

once again, $\mathcal{A}_A$ is a matrix of the form 

$ \begin{pmatrix}
1 & a & b\\
0 & 1 & a\\
0 & 0 & 1
\end{pmatrix} $

where $ a_A = \begin{pmatrix}
a & b & c
\end{pmatrix} $

you will have to take the transpose, inv transpose, or inv of some things to make it work right


# notes
observer gain $ L_o $ that places the poles at the desired locations is given by: $ A_o − L_o C_o $ but this is only when in observer canonic form

`o` subscript gains mean the array is in observer canonical form